# Overview of [smolagents](https://huggingface.co/docs/smolagents)

In [1]:
import os

import torch
from sentence_transformers import SentenceTransformer
from smolagents import LiteLLMModel, ToolCallingAgent

from travel_agent.qdrant import client as qdrant_client
from travel_agent.retrieval.smolagents.tool import TravelReviewQueryTool

if not os.getenv("QDRANT_URL"):
    print("Set QDRANT_URL and QDRANT_API_KEY env")

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else ("mps" if torch.backends.mps.is_available() else "cpu"))
device

device(type='mps')

In [3]:
# Replace with your favourite local model
MODEL_NAME = "hf.co/IlyaGusev/saiga_nemo_12b_gguf:Q4_0"
model_id = f"ollama_chat/{MODEL_NAME}"

collection_name = "moskva_cointegrated_rubert_tiny2"
embed_model_name = "cointegrated/rubert-tiny2"


model = LiteLLMModel(
    model_id=model_id,
    api_base="http://127.0.0.1:11434",
)

embed_model = SentenceTransformer(embed_model_name, device=device)

agent = ToolCallingAgent(
    tools=[TravelReviewQueryTool(embed_model, qdrant_client, collection_name)], model=model, max_steps=2
)

2025-04-19 19:33:05.739 | DEBUG    | travel_agent.qdrant.client:create_client:12 - Connecting to Qdrant at https://21a1c8c4-0d63-4176-9930-ee7848dafd2f.eu-central-1-0.aws.cloud.qdrant.io
2025-04-19 19:33:06.685 | SUCCESS  | travel_agent.qdrant.client:create_client:22 - connected: title='qdrant - vector search engine' version='1.13.6' commit='4db98ecdbf66bedbce268ec26865ea446d0bba7b'
No sentence-transformers model found with name cointegrated/rubert-tiny2. Creating a new one with mean pooling.


In [4]:
print(agent.system_prompt)

You are an expert assistant who can solve any task using tool calls. You will be given a task to solve as best you can.
To do so, you have been given access to some tools.

The tool call you write is an action: after the tool is executed, you will get the result of the tool call as an "observation".
This Action/Observation can repeat N times, you should take several steps when needed.

You can use the result of the previous action as input for the next action.
The observation will always be a string: it can represent a file, like "image_1.jpg".
Then you can use it as input for the next action. You can do it for instance as follows:

Observation: "image_1.jpg"

Action:
{
  "name": "image_transformer",
  "arguments": {"image": "image_1.jpg"}
}

To provide the final answer to the task, use an action blob with "name": "final_answer" tool. It is the only way to complete the task, else you will be stuck on a loop. So your final output should look like this:
Action:
{
  "name": "final_answer"

In [14]:
# Sanity check
agent.run("Что ты думаешь о римской империи?")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Что ты думаешь о римской империи?                                                                               │
│                                                                                                                 │
╰─ LiteLLMModel - ollama_chat/hf.co/IlyaGusev/saiga_nemo_12b_gguf:Q4_0 ───────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'final_answer' with arguments: {'answer': 'Римская империя была одной из самых влиятельных и      │
│ обширных государственных образований в истории человечества. Вот краткий обзор её достижений и особенностей:\n- │
│ Образовалась около 27 года до н.э.;\n- Простиралась на огромную территорию от Великобритании до Египта и от     │
│ Испании до Сирии;\n- Развивала законодательство, культуру и архитектуру; при Риме были построены многие         │
│ известные памятники и монументы, такие как Колизей в Риме или Пантеон.\n- Внесла значительный вклад в право     │
│ (законы 12 таблиц), образование (римские школы и университеты) и язык (латинский стал международным языком      │
│ общения);\n- Управлялась через строгую политическую структуру, включая сенат и диктаторский режим;\n- Время её  │
│ расцвета приходится на 1 век н.э., известное как '}                                                             │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Final answer: Римская империя была одной из самых влиятельных и обширных государственных образований в истории 
человечества. Вот краткий обзор её достижений и особенностей:
- Образовалась около 27 года до н.э.;
- Простиралась на огромную территорию от Великобритании до Египта и от Испании до Сирии;
- Развивала законодательство, культуру и архитектуру; при Риме были построены многие известные памятники и 
монументы, такие как Колизей в Риме или Пантеон.
- Внесла значительный вклад в право (законы 12 таблиц), образование (римские школы и университеты) и язык 
(латинский стал международным языком общения);
- Управлялась через строгую политическую структуру, включая сенат и диктаторский режим;
- Время её расцвета приходится на 1 век н.э., известное как 

[Step 1: Duration 33.91 seconds| Input tokens: 1,128 | Output tokens: 231]

'Римская империя была одной из самых влиятельных и обширных государственных образований в истории человечества. Вот краткий обзор её достижений и особенностей:\n- Образовалась около 27 года до н.э.;\n- Простиралась на огромную территорию от Великобритании до Египта и от Испании до Сирии;\n- Развивала законодательство, культуру и архитектуру; при Риме были построены многие известные памятники и монументы, такие как Колизей в Риме или Пантеон.\n- Внесла значительный вклад в право (законы 12 таблиц), образование (римские школы и университеты) и язык (латинский стал международным языком общения);\n- Управлялась через строгую политическую структуру, включая сенат и диктаторский режим;\n- Время её расцвета приходится на 1 век н.э., известное как '

In [5]:
agent.run("Я бы хотел посетить японский ресторан. Можешь ли посоветовать хороший в Москве?")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Я бы хотел посетить японский ресторан. Можешь ли посоветовать хороший в Москве?                                 │
│                                                                                                                 │
╰─ LiteLLMModel - ollama_chat/hf.co/IlyaGusev/saiga_nemo_12b_gguf:Q4_0 ───────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'travel_review_query' with arguments: {'query': 'Лучшие японские рестораны в Москве'}             │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Observations: Найденные отзывы о местах:

=== Отзыв на Lou lou ===
Адрес: Москва, Малая Бронная улица, 10, стр. 1
Рейтинг: 5.0
Категории: Ресторан
Отзыв: Любимый ресторан 😻коктейли лучшие в Москве.  

=== Отзыв на Geraldine ===
Адрес: Москва, улица Остоженка, 27, корп. 2
Рейтинг: 5.0
Категории: Ресторан;Доставка еды и обедов
Отзыв: Один из лучших ресторанов Москвы !  

=== Отзыв на Vice Burgers ===
Адрес: Москва, Новодмитровская улица, 2к2
Рейтинг: 5.0
Категории: Кафе;Быстрое питание
Отзыв: Самые вкусные бургеры в Москве за последнее время. 

=== Отзыв на Burger Heroes ===
Адрес: Москва, проспект Мира, 119, стр. 56
Рейтинг: 5.0
Категории: Быстрое питание;Бар, паб;Кафе
Отзыв: Лучшая бургерная Москвы! 

=== Отзыв на Неаполь ===
Адрес: Москва, Вознесенский переулок, 12, стр. 3
Рейтинг: 5.0
Категории: Гостиница
Отзыв: Хорошая уютная гостиница в центре Москвы

[Step 1: Duration 22.58 seconds| Input tokens: 1,142 | Output tokens: 32]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'final_answer' with arguments: {'answer': 'Наиболее высоко оцененными ресторанами с отличными     │
│ отзывами являются Lou lou и Geraldine. Выбирайте между:\n\n- **Lou lou** на улице Малая Бронная, 10, стр. 1     │
│ (рейтинг: 5.0) - особенно отмечен за лучшие коктейли в Москве.\n- **Geraldine** по адресу улица Остоженка, 27,  │
│ корп. 2 (рейтинг: 5.0) - один из лучших ресторанов Москвы.\nОба варианта заслуживают внимания.'}                │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Final answer: Наиболее высоко оцененными ресторанами с отличными отзывами являются Lou lou и Geraldine. Выбирайте 
между:

- **Lou lou** на улице Малая Бронная, 10, стр. 1 (рейтинг: 5.0) - особенно отмечен за лучшие коктейли в Москве.
- **Geraldine** по адресу улица Остоженка, 27, корп. 2 (рейтинг: 5.0) - один из лучших ресторанов Москвы.
Оба варианта заслуживают внимания.

[Step 2: Duration 30.23 seconds| Input tokens: 2,777 | Output tokens: 183]

'Наиболее высоко оцененными ресторанами с отличными отзывами являются Lou lou и Geraldine. Выбирайте между:\n\n- **Lou lou** на улице Малая Бронная, 10, стр. 1 (рейтинг: 5.0) - особенно отмечен за лучшие коктейли в Москве.\n- **Geraldine** по адресу улица Остоженка, 27, корп. 2 (рейтинг: 5.0) - один из лучших ресторанов Москвы.\nОба варианта заслуживают внимания.'